<a href="https://colab.research.google.com/github/Hamoda-dabbit/Mining---classification-in-Arabic-Article/blob/main/RandomForest/RandomForest_TFidf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 1-import library
 
  إستيراد المكتبات

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import nltk 
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem.isri import ISRIStemmer
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from joblib import dump, load

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


--------

2- import Datasets

استيراد ملفات البيانات


In [ ]:
# df = pd.read_excel("drive/MyDrive/mining/1.xlsx")
# df = pd.read_excel("drive/MyDrive/mining/2.xlsx")
# df = pd.read_excel("drive/MyDrive/mining/3.xlsx")
df = pd.read_excel("drive/MyDrive/mining/4.xlsx")

------
3- shuffle  
بعثرة البيانات

In [ ]:
df=shuffle(df)
df

,text,targe
67806,الحواصلي قال إن خوض مباراة في موسم واحد دليل ع...,4
59635,تقرير مجلس المنافسة يؤكد أن أربع شركات تهيمن ع...,2
7593,كافي توبا مشروب قهوة يحظى بشعبية منقطعة النظير...,2
52567,علمت الصباح »، أن عملا فنيا من المتوقع أن يجمع...,0
42566,بعضها يلجأ إلى أسلوب التحايل لتحويل مزيد من ال...,2
...,...,...
33575,أقصي فريق أولمبيك خريبكة من الدور الأول لمسابق...,4
40423,المتهم ستيني قرر قضاء ليلة معهما قبل أن يعرضهم...,1
65192,كشفت صحيفة دير شبيغل الألمانية الجمعة أن فوز ا...,4
16681,ختتم مساء اليوم الثلاثاء عروض المسابقة الرسمية...,0


------
4- clean data

تنظيف البيانات

In [ ]:
print(np.sum(df.isnull().any(axis=1)))

clean_df= df.dropna()
print(np.sum(clean_df.isnull().any(axis=1)))
#-------------
print (df.notnull().any(axis = 0))
print(len(df))
#-------------
def remove_hashtag(df, col = 'text'):
    for letter in r'#.][!XR':
      df[col] = df[col].astype(str).str.replace(letter,'', regex=True)
        
remove_hashtag(clean_df)
clean_df
#-------------
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)
#-------------
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text
#-------------    
def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)
#-------------
def processDocument(doc, stemmer): 

    #Replace @username with empty string
    doc = re.sub(r'@[^\s]+', ' ', doc)
    doc = re.sub(r'_', ' ', doc)
    doc = re.sub(r'\n', ' ', doc)
    doc = re.sub(r'[a-z,A-Z]', '', doc)
    doc = re.sub(r'\d', '', doc)
    #Convert www.* or https?://* to " "
    doc = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',doc)
    #Replace #word with word
    doc = re.sub(r'#([^\s]+)', r'\1', doc)
    # remove punctuations
    doc= remove_punctuations(doc)
    # normalize the tweet
    doc= normalize_arabic(doc)
    # remove repeated letters
    doc=remove_repeating_char(doc)
    #stemming
    doc = stemmer.stem(doc)
    
    return doc

stemmer = ISRIStemmer()
clean_df["text"] = clean_df['text'].apply(lambda x: processDocument(x, stemmer))
print(clean_df)

378
0
text     True
targe    True
dtype: bool
68690
                                                    text  targe
67806  حواصلي قال ان خوض مباراه في موسم واحد دليل علي...      4
59635  تقرير مجلس المنافسه يؤكد ان اربع شركات تهيمن ع...      2
7593   كافي توبا مشروب قهوه يحظي بشعبيه منقطعه النظير...      2
52567  علمت الصباح » ان عملا فنيا من المتوقع ان يجمع ...      0
42566  بعضها يلجا الي اسلوب التحايل لتحويل مزيد من ال...      2
...                                                  ...    ...
33575  اقصي فريق اولمبيك خريبكه من الدور الاول لمسابق...      4
40423  متهم ستيني قر قضاء ليله معهما قبل ان يعرضهما ل...      1
65192  كشفت صحيفه دير شبيغل الالمانيه الجمعه ان فوز ا...      4
16681  ختم مساء اليوم الثلاثاء عروض المسابقه الرسميه ...      0
11311  اعلنت سفاره المغرب في كوت ديفوار اليوم الاحد ا...      3

[68312 rows x 2 columns]


----
5-
Tokinaizer: TFidf

تحويل النصوص إلى أرقام عبر خوازرمية
TfidfVectorizer




In [ ]:
y = clean_df['targe']
X = clean_df['text']
stopwords_list = stopwords.words('arabic')

tfidf = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7,token_pattern=r"(?u)\b\w\w+\b",stop_words=stopwords_list)

y1 = clean_df['targe']
X1 = clean_df['text']
X1= tfidf.fit_transform (X1) .toarray ()


----
6- accuracy calculation

عمل تابع لحساب الدقة

In [ ]:
def calculate_results(y_true, y_pred):
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

---------------------
7- classifier:RandomForestClassifier

with Cross validation


التنصنيف والتدريب عبر خوارزمية

RandomForestClassifier


استخدام خوارزمية

Cross validation

لتوزيع عينات الاختبار والتأكد من عدم وجود 

Overfitting

In [ ]:
from sklearn.model_selection import StratifiedKFold

from sklearn.ensemble import RandomForestClassifier

strtfdKFold = StratifiedKFold(n_splits=5)
kfold = strtfdKFold.split(X1, y1)
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
result_RandomForestClassifier=[]
#
for k, (train_index, test_index) in enumerate(kfold):
    X_train, X_test = X1[train_index], X1[test_index]
    y_train, y_test = y1.iloc[train_index], y1.iloc[test_index]
    classifier.fit(X_train, y_train)
    y_pred=(classifier.predict(X_test))
    result_RandomForestClassifier.append(calculate_results(y_test, y_pred))


حساب متوسط النتائج للـ5 مراحل في cross validation 

In [ ]:
accuracy=0
f1=0
precision=0
recall=0
for i in result_RandomForestClassifier:
  accuracy+=(i['accuracy'])
  f1+=(i['f1'])
  precision+=(i['precision'])
  recall+=(i['recall'])

accuracy=accuracy/len(result_RandomForestClassifier)
f1=f1/len(result_RandomForestClassifier)
precision=precision/len(result_RandomForestClassifier)
recall=recall/len(result_RandomForestClassifier)
# (result_DecisionTreeClassifier)
results_RandomForestClassifier=[{'accuracy':accuracy,'f1':f1,'precision':precision,'recall':recall}]
results_RandomForestClassifier

[{'accuracy': 92.80506484204825,
  'f1': 0.9280963000587084,
  'precision': 0.9283160067681553,
  'recall': 0.9280506484204825}]

-----------------------
-----------------------
-----------------------
8-Results

 النتائج

نتائج التدريب لكل قاعدة بيانات

In [ ]:
print('DecisionTreeClassifier results in dataset 1:')
results_RandomForestClassifier

DecisionTreeClassifier results in dataset 1:


[{'accuracy': 94.79871052663006,
  'f1': 0.948054790221879,
  'precision': 0.9484085086796259,
  'recall': 0.9479871052663006}]

In [ ]:
print('DecisionTreeClassifier results in dataset 2:')
results_RandomForestClassifier

DecisionTreeClassifier results in dataset 2:


[{'accuracy': 94.35977276279998,
  'f1': 0.9436010724434535,
  'precision': 0.9437771120290449,
  'recall': 0.9435977276279998}]

In [ ]:
print('DecisionTreeClassifier results in dataset 3:')
results_RandomForestClassifier

DecisionTreeClassifier results in dataset 3:


[{'accuracy': 93.20200229777969,
  'f1': 0.9320751779523017,
  'precision': 0.9322813561648182,
  'recall': 0.9320200229777968}]

In [3]:
print('DecisionTreeClassifier results in dataset 4:')
results_RandomForestClassifier

DecisionTreeClassifier results in dataset 4:


[{'accuracy': 92.80506484204825,
  'f1': 0.9280963000587084,
  'precision': 0.9283160067681553,
  'recall': 0.9280506484204825}]

In [4]:
Final_Result=[
  {'accuracy': 94.79871052663006,
  'f1': 0.948054790221879,
  'precision': 0.9484085086796259,
  'recall': 0.9479871052663006}         
  ,
  {'accuracy': 94.35977276279998,
  'f1': 0.9436010724434535,
  'precision': 0.9437771120290449,
  'recall': 0.9435977276279998}
  ,
  {'accuracy': 93.20200229777969,
  'f1': 0.9320751779523017,
  'precision': 0.9322813561648182,
  'recall': 0.9320200229777968}
  ,
  {'accuracy': 92.80506484204825,
  'f1': 0.9280963000587084,
  'precision': 0.9283160067681553,
  'recall': 0.9280506484204825}  
  ]

accuracy=0
f1=0
precision=0
recall=0
for i in Final_Result:
  accuracy+=(i['accuracy'])
  f1+=(i['f1'])
  precision+=(i['precision'])
  recall+=(i['recall'])

accuracy=accuracy/len(Final_Result)
f1=f1/len(Final_Result)
precision=precision/len(Final_Result)
recall=recall/len(Final_Result)
# (result_DecisionTreeClassifier)
Final_Result=[{'accuracy':accuracy,'f1':f1,'precision':precision,'recall':recall}]

print('Final_Result:')
print(Final_Result)

Final_Result:
[{'accuracy': 93.79138760731449, 'f1': 0.9379568351690857, 'precision': 0.9381957459104111, 'recall': 0.9379138760731449}]
